In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Flatten, SimpleRNN, LSTM, Input, GRU, InputLayer, GlobalMaxPooling1D
from tensorflow.keras.optimizers import Adam, SGD
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

/kaggle/input/cs985-cs987-goodread-class-project/my_goodreads_test.csv
/kaggle/input/cs985-cs987-goodread-class-project/my_goodreads_train.csv
/kaggle/input/cs985-cs987-goodread-class-project/my_goodreads_sample_submission.csv


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
test_set = pd.read_csv('/kaggle/input/cs985-cs987-goodread-class-project/my_goodreads_test.csv')
train_set = pd.read_csv('/kaggle/input/cs985-cs987-goodread-class-project/my_goodreads_train.csv')
train_set.head(1)

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,1da3dcc13cf360b9ecb06251e86237535,182091851,1c949b5dedf860514ec8b97ad158c9f4f,5,I thought this was a very good historical fant...,Fri Feb 01 16:44:16 -0800 2013,Tue Oct 25 18:49:25 -0700 2016,Sat May 04 00:00:00 -0700 2013,Thu Apr 25 00:00:00 -0700 2013,5,6


In [3]:
train_set = train_set[:200000] 

In [4]:
stop = stopwords.words('english')
# remove stop words from train and test set
train_set['review_text'] = train_set['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_set['review_text'] = test_set['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [5]:
# fit the tokenizer on the train set
vocab_size = 5000
tok = Tokenizer(num_words=vocab_size, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tok.fit_on_texts(train_set['review_text'].values)
wi = tok.word_index
print(len(wi))

170832


In [6]:
def process_dataset(train_df, test_df):
    """Produce the train set, validation set, eval set and test set

    Args:
        train_df (pd.DataFrame): initial train set
        test_df (pd.DataFrame): initial test set

    Returns:
        tuple: all relevant datasets
    """
    # Tokenize the initial train set and categorise labels
    X = tok.texts_to_sequences(train_df['review_text'].values)
    X = pad_sequences(X, maxlen=250)
    y = to_categorical(train_df['rating'], num_classes=6)
    # Split eval set and train set
    X_eval, y_eval = X[-100001:], y[-100001:]
    X, y = X[:-100000], y[:-100000]
    # Split train set and validation set
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
    # Tokenize initial test set
    X_test = tok.texts_to_sequences(test_df['review_text'].values)
    X_test = pad_sequences(X_test, maxlen=250)
    return X_train, X_valid, X_eval, X_test, y_train, y_valid, y_eval 

In [7]:
X_train, X_valid, X_eval, X_test, y_train, y_valid, y_eval = process_dataset(train_set, test_set)

In [8]:
GRU_model = Sequential([
    Embedding(vocab_size + 1, 64, input_length = 250),
    GRU(32, return_sequences=False),
    Dense(16, activation='tanh'),
    Dropout(0.5),
    Dense(6, activation='softmax')] # 6 output nodes numbers 0-5
                       )

GRU_model.compile(Adam(learning_rate=1E-4),
              loss = 'categorical_crossentropy', 
              metrics=['accuracy'])

In [9]:
GRU_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 250, 64)           320064    
                                                                 
 gru (GRU)                   (None, 32)                9408      
                                                                 
 dense (Dense)               (None, 16)                528       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 102       
                                                                 
Total params: 330,102
Trainable params: 330,102
Non-trainable params: 0
_________________________________________________________________


In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)
# fit model with early stopping callback
GRU_history = GRU_model.fit(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid), callbacks=[callback])

Epoch 1/5
2500/2500 [==============================] - 171s 67ms/step - loss: 1.5124 - accuracy: 0.3261 - val_loss: 1.3112 - val_accuracy: 0.4259
Epoch 2/5
2500/2500 [==============================] - 168s 67ms/step - loss: 1.3103 - accuracy: 0.4180 - val_loss: 1.2123 - val_accuracy: 0.4697
Epoch 3/5
2500/2500 [==============================] - 169s 68ms/step - loss: 1.2268 - accuracy: 0.4602 - val_loss: 1.1688 - val_accuracy: 0.4859
Epoch 4/5
2500/2500 [==============================] - 169s 67ms/step - loss: 1.1814 - accuracy: 0.4867 - val_loss: 1.1420 - val_accuracy: 0.4982
Epoch 5/5
2500/2500 [==============================] - 169s 68ms/step - loss: 1.1441 - accuracy: 0.5096 - val_loss: 1.1266 - val_accuracy: 0.5074


In [11]:
evaluation_results = GRU_model.evaluate(X_eval, y_eval)

3126/3126 [==============================] - 64s 20ms/step - loss: 1.1295 - accuracy: 0.5058


In [12]:
predictions = GRU_model.predict(X_test)

6250/6250 [==============================] - 124s 20ms/step


In [13]:
def submit_predictions(results):
    final = np.argmax(results, axis=1)
    submission_df = test_set[['review_id']]
    submission_df['rating'] = final.astype(int)
    submission_df.to_csv('submission.csv', index=False)
    return submission_df

In [14]:
submit_predictions(predictions)

/tmp/ipykernel_20/2102143267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_df['rating'] = final.astype(int)


,review_id,rating
0,16c32b9bb0142734c60f8b028f281c5dd,3
1,1f86ec340856f1097c922d33aed48d4ea,5
2,14a20b34f27611e45bd75e24de6a06324,5
3,1b2e5b069e4f440d152ae6d48e3f111ac,4
4,132aec4dfb13a2f8032a7431023dd728f,3
...,...,...
199995,1e65b0ab60fb0fc5361da7c3a7a45bc2a,4
199996,176c31b9581191138cc477b0df2f82b09,3
199997,17af2bb1902e2b7f6a5004e7bbe285139,5
199998,1e13fa3ce7e49dfd190bd0a02c36efe61,4
